In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('Customer-Churn-Records.csv')
# print(df.head())
# print(df.columns)
# print(df['Age'])
# print(df.shape)
# Explore the dataset
# print("=== DATASET EXPLORATION ===")
# print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
print("\nTarget Distribution:")
print(df['Exited'].value_counts())

# Visualize the target variable
plt.figure(figsize=(8, 5))
df['Exited'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Customer Churn Distribution (0 = Stay, 1 = Leave)')
plt.xlabel('Churned')
plt.ylabel('Count')
plt.show()
# Data Cleaning & Feature Engineering
df_clean = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Convert categorical variables
df_clean = pd.get_dummies(df_clean, columns=['Geography', 'Gender', 'Card Type'], drop_first=True)

# Check the processed data
print("Processed Data Shape:", df_clean.shape)
print("Columns after preprocessing:")
print(df_clean.columns.tolist())

# Prepare features and target
X = df_clean.drop('Exited', axis=1)
y = df_clean['Exited']

print(f"Features: {X.shape[1]}, Target: {y.shape[0]}")
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Initialize and train Logistic Regression
print("\n=== TRAINING LOGISTIC REGRESSION ===")
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores

print("✅ Model training completed!")
# Model Evaluation
print("=== MODEL EVALUATION ===")

accuracy = accuracy_score(y_test, y_pred)
print(f"📊 Accuracy: {accuracy:.4f}")

print("\n📈 Detailed Classification Report:")
print(classification_report(y_test, y_pred))

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': abs(model.coef_[0])  # Absolute value of coefficients
}).sort_values('importance', ascending=False)

print("\n🔍 Top 10 Most Important Features:")
print(feature_importance.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
plt.title('Top 10 Feature Importance - Logistic Regression')
plt.tight_layout()
plt.show()
# Advanced Insights
print("=== ADVANCED ANALYSIS ===")

# Confidence scores for first 10 predictions
print("\n🎯 Prediction Confidence (First 10 customers):")
for i in range(10):
    actual = y_test.iloc[i]
    predicted = y_pred[i]
    confidence = y_pred_proba[i]
    status = "✅ Correct" if predicted == actual else "❌ Wrong"
    print(f"Customer {i+1}: Predicted {predicted}, Actual {actual}, Confidence: {confidence:.3f} {status}")

# Business Impact Analysis
churn_rate = y_test.mean()
print(f"\n💼 Business Insights:")
print(f"Churn Rate in Test Set: {churn_rate:.2%}")
print(f"Model detected {y_pred.sum()} out of {y_test.sum()} actual churns")

# Which customers are most likely to churn?
high_risk_customers = X_test[y_pred_proba > 0.7]  # High probability of churning
print(f"\n🚨 High-Risk Customers (>{70}% churn probability): {len(high_risk_customers)}")
# Experiment with different approaches
print("\n=== EXPERIMENTATION ===")

# Try with feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model_scaled = LogisticRegression(random_state=42, max_iter=1000)
model_scaled.fit(X_train_scaled, y_train)
y_pred_scaled = model_scaled.predict(X_test_scaled)

accuracy_scaled = accuracy_score(y_test, y_pred_scaled)
print(f"📊 Accuracy with Scaling: {accuracy_scaled:.4f}")
print(f"📈 Improvement: {accuracy_scaled - accuracy:.4f}")